<a href="https://colab.research.google.com/github/electricalgorithm/div2k_realesrgan_hat_trainer/blob/main/Real_ESRGAN_with_DIV2K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preperation of the Environment & Dataset

**Dowload the original Real-ESRGAN codebase.**

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git

**Install dependencies.**

In [ ]:
%cd Real-ESRGAN
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
%cd ..

**Download DIV2K dataset.**

In [ ]:
!mkdir /content/Real-ESRGAN/datasets
%cd /content/Real-ESRGAN/datasets
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!unzip DIV2K_train_HR.zip
%cd /content/

**Generate multiscale images.**

In [ ]:
!mkdir /content/Real-ESRGAN/datasets/DIV2K_multiscale/
%cd /content/Real-ESRGAN/scripts
!python generate_multiscale_DF2K.py --input /content/Real-ESRGAN/datasets/DIV2K_train_HR --output /content/Real-ESRGAN/datasets/DIV2K_multiscale
%cd /content/

**Generate sub-images.**

In [ ]:
%cd /content/Real-ESRGAN/scripts
!python extract_subimages.py --input /content/Real-ESRGAN/datasets/DIV2K_multiscale --output /content/Real-ESRGAN/datasets/DIV2K_multiscale_sub --crop_size 400 --step 200
%cd /content/

**Combine all the directories inside a root folder.**


In [ ]:
!echo "Total Image Count on Multiscale-sub: "
!ls -alt /content/Real-ESRGAN/datasets/DIV2K_multiscale_sub | wc -l

!echo "Total Image Count on Multiscale: "
!ls -alt /content/Real-ESRGAN/datasets/DIV2K_multiscale | wc -l


!echo "Total Image Count on original HR: "
!ls -alt /content/Real-ESRGAN/datasets/DIV2K_train_HR | wc -l

Total Image Count on Multiscale-sub: 
45764
Total Image Count on Multiscale: 
3203
Total Image Count on original HR: 
803


In [ ]:
!mkdir /content/Real-ESRGAN/datasets/DIV2K
!cp -r /content/Real-ESRGAN/datasets/DIV2K_multiscale_sub /content/Real-ESRGAN/datasets/DIV2K
!cp -r /content/Real-ESRGAN/datasets/DIV2K_multiscale /content/Real-ESRGAN/datasets/DIV2K
!cp -r /content/Real-ESRGAN/datasets/DIV2K_train_HR /content/Real-ESRGAN/datasets/DIV2K
!rm -rf /content/Real-ESRGAN/datasets/DIV2K_multiscale_sub
!rm -rf /content/Real-ESRGAN/datasets/DIV2K_multiscale
!rm -rf /content/Real-ESRGAN/datasets/DIV2K_train_HR

**Generate metadata text-file.**

In [ ]:
%cd /content/Real-ESRGAN/
!python scripts/generate_meta_info.py --input datasets/DIV2K/DIV2K_train_HR datasets/DIV2K/DIV2K_multiscale --root datasets/DIV2K datasets/DIV2K --meta_info datasets/DIV2K/meta_info/meta_info_DIV2Kmultiscale.txt
%cd ..

# Preperation for Training

**Download ESRGAN model.**

In [ ]:
%cd /content/Real-ESRGAN/
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/ESRGAN_SRx4_DF2KOST_official-ff704c30.pth -P experiments/pretrained_models
%cd ..

**Change the config within new dataset.**

In [ ]:
!cp /content/Real-ESRGAN/options/train_realesrnet_x4plus.yml /content/Real-ESRGAN/options/train_custom_dataset.yml
# Update dataset location in the new train_custom_dataset.yml.

**Test if everything works for a small period of time.**

In [ ]:
%cd /content/Real-ESRGAN/
!python realesrgan/train.py -opt options/train_custom_dataset.yml --debug
%cd ..

# Training Process

In [ ]:
%cd /content/Real-ESRGAN/
!python realesrgan/train.py -opt options/train_custom_dataset.yml --auto_resume
%cd ..